# Homogeneous Cube

The purposes of this notebook are the following

<ol>
    <li>OpenMC comparison with Reactor Theory for homogeneous reactors with specified composition - find critical size</li>
    <li>Experiment with distributed materials for burn-up calculations</li>
</ol>


    


In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import openmc

## Diffusion Theory / Reactor Theory Problem Specification

Find the critical size of a homogeneous cube reactor comprising a mixture of U-235 and light water in which the light water is present at a given weight percent.  The density of the Uranium added to the mixture is 19.1 g/cc; the density of the light water is 1 g/cc.  The mixture temperature is 20 $^{\circ}$C. 

In [2]:
Nf_w_o = 0.03; # weight percent of fuel
Nm_w_o = 1-Nf_w_o; # weight percent of moderator

rho_f = 19.1; # g/cc, density of fuel
rho_m = 1.0; # g/cc, density of moderator

rho_mix = (Nf_w_o/rho_f + Nm_w_o/rho_m)**(-1); # density of mixture

print('The mixture density is approximately %6.5f g/cc.' % rho_mix);


The mixture density is approximately 1.02926 g/cc.


In [3]:
M_f = 235; # g/mol, gram atomic weight of U-235 fuel
M_m = 18; # g/mol, gram atomic weight of light water moderator

Nf = Nf_w_o*rho_mix/M_f*0.6022; # atom/b-cm, atom density of fuel
Nm = Nm_w_o*rho_mix/M_m*0.6022; # atom/b-cm, atom density of the moderator

print('Atom density of fuel: %6.5g atom/b-cm.' % Nf);
print('Atom density of the moderator: %6.5g atom/b-cm.' % Nm);

Atom density of fuel: 7.9126e-05 atom/b-cm.
Atom density of the moderator: 0.033401 atom/b-cm.


### Find Z

For Reactor Theory calculations where the composition (atom density of fuel and moderator) is specified, we will calculate the parameter Z which is the ratio of the thermal average macroscopic absorption cross section for the fuel and moderator respectively:

$$ Z = \frac{\bar{\Sigma}_{a_{F}}}{\bar{\Sigma}_{a_{M}}}$$

In [4]:
# from Table II.2 and II.3 in LaMarsh
sigma_o_a_F = 687; #b, absorption cross section of U-235 at 0.0253 eV
sigma_o_a_M = 0.664; #b, absorption cross section of light water at 0.0253 eV

To = 293.; # K, reference temperature for cross sections
T = 293.; # K, system temperature

# from Table 3.2 in LaMarsh
g_a_F = 0.9780; # non 1/v factor for fuel

# thermal average macroscopic cross sections (1/cm)
Sigma_bar_a_F = Nf*((np.pi)/2)**(0.5)*(To/T)**(0.5)*g_a_F*sigma_o_a_F;
Sigma_bar_a_M = Nm*((np.pi)/2)**(0.5)*(To/T)**(0.5)*sigma_o_a_M;

Z = Sigma_bar_a_F/Sigma_bar_a_M;

print('Z = %6.5g.' % Z);


Z = 2.3971.


### Find $k_{\infty}$

For reactor theory, $k_{\infty}=\eta_{T}f$ and $f = \frac{Z}{Z+1}$

In [5]:
eta_T = 2.065; # from Table 6.3 of LaMarsh
f = Z/(Z+1.);

k_inf = eta_T*f;

print('k-infinity = %6.5g.' % k_inf)

k-infinity = 1.4571.


### Find Migration Area

From Diffusion Theory, $M_T^2 = L_T^2 + \tau_T$. Where $L^2_T$ is the thermal diffusion area and $\tau_T$ is the Fermi age.

$$L_T^2 = \frac{L^2_{T_M}}{Z+1}$$
$$\tau \approx \tau_{T_{M}}$$

In [6]:
diff_area_mod = 8.1; # cm^2, from Table 5.2 LaMarsh
tau_t_mod = 27; # cm, from Table 5.3 LaMarsh

therm_diff_area = diff_area_mod/(Z+1);
migration_area = therm_diff_area + tau_t_mod; # cm^2
print('Thermal Migration Area = %6.5g cm^2.' % migration_area)

Thermal Migration Area = 29.384 cm^2.


### Find Buckling

From Modified 1-group Reactor Theory, buckling is given by:
$$B^2 = \frac{k_{\infty}-1}{M^2_T}$$

In [7]:
B_sqr = (k_inf - 1.)/migration_area; # cm^-2, buckling

print('Buckling = %6.5g cm^-2.' % B_sqr); 

Buckling = 0.015557 cm^-2.


### Find Side-length of Cube

From Diffusion Theory, the buckling for a cube is given by:
$$B^2 = 3\left(\frac{\pi}{a} \right)^2$$
where $a$ is the side length of the reactor.

In this case, buckling is known, so we solve for the side length:

$$a = \left[\frac{3\pi^2}{B^2} \right]^{\frac{1}{2}}$$

In [8]:
a = (3*(np.pi**2)/(B_sqr))**(0.5); # cm, side length

print('Reactor side-length = %6.3g cm' % a);

Reactor side-length =   43.6 cm


## OpenMC Model

In this part of the notebook, we will develop an OpenMC model with the same material composition.  The geometry will be parameterized by side-length and we will use the criticality search tools of OpenMC to find the critical size.

We will also exploit the symmetry of the problem and only model 1/8th of the core (upper corner cube).  This will (hopefully) pay dividends when it comes to keeping reaction rate tallies and ultimately a discretized verion of the core to track material changes with burnup.

### OpenMC Model Function

In [9]:
def build_model(a):

    fuel = openmc.Material();
    fuel.add_nuclide("U235",1,percent_type="ao")
    fuel.set_density("g/cc",19.1);

    mod =  openmc.Material();
    mod.add_element("H",2,percent_type="ao")
    mod.add_element("O",1,percent_type="ao")
    mod.set_density("g/cc",1.0);


    # use the cool classmethod mix_materials to create a
    # new material from a mixture of existing materials.
    core_mat = openmc.Material.mix_materials([fuel,mod],
                                     [Nf_w_o,Nm_w_o],
                                     percent_type='wo');
    core_mat.add_s_alpha_beta('c_H_in_H2O');
    
    materials = openmc.Materials([core_mat]);
    
    # geometry
    bottom = openmc.ZPlane(z0=0,boundary_type='reflective');
    top = openmc.ZPlane(z0=a/2.,boundary_type='vacuum');
    front = openmc.XPlane(x0=a/2.,boundary_type='vacuum');
    back = openmc.XPlane(x0=0.,boundary_type='reflective');
    left = openmc.YPlane(y0=0.,boundary_type='reflective');
    right=openmc.YPlane(y0=a/2.,boundary_type='vacuum');
    
    # core cell
    core_cell = openmc.Cell();
    core_cell.fill = core_mat;
    core_cell.region = +bottom & -top & -front & +back & +left & -right
    
    # root universe
    root_universe = openmc.Universe()
    root_universe.add_cells([core_cell]);
    
    geometry = openmc.Geometry(root_universe);
    
    settings = openmc.Settings()
    
    settings.batches = 200;
    settings.inactive = 50;
    settings.particles = 5000;
    
    bounds = [0,0,0,a/2.,a/2.,a/2.];
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                    only_fissionable=True);
    settings.source = openmc.Source(space=uniform_dist);
    
    # okay - no disk output for now
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(geometry,materials,settings)
    
    return model
    
    
    

In [ ]:
crit_size, guesses, keffs = openmc.search_for_keff(build_model,bracket=[20,500],
                                                  tol=1e-3,print_iterations=True)

print('Critical Cube Side-Length: %6.4f cm.' % crit_size)

Iteration: 1; Guess of 2.00e+01 produced a keff of 0.50394 +/- 0.00098
Iteration: 2; Guess of 5.00e+02 produced a keff of 1.44505 +/- 0.00090
Iteration: 3; Guess of 2.60e+02 produced a keff of 1.43252 +/- 0.00090
Iteration: 4; Guess of 1.40e+02 produced a keff of 1.39640 +/- 0.00101
Iteration: 5; Guess of 8.00e+01 produced a keff of 1.29949 +/- 0.00099
Iteration: 6; Guess of 5.00e+01 produced a keff of 1.12674 +/- 0.00115
Iteration: 7; Guess of 3.50e+01 produced a keff of 0.92087 +/- 0.00106
Iteration: 8; Guess of 4.25e+01 produced a keff of 1.04028 +/- 0.00103
